## Health Condition using 'hcond' and 'hcondnew'
### By Gavin Qu - May 23rd 2024
#### Data Extraction 

-	Encode hcond and new healthcond variables correctly 
-	Note that individuals are asked about pre-existing health conditions on their first interview in the UKHLS – the hcond (i) variable, where i codes different conditions – and then asked whether they have developed new conditions in subsequent interviews – the hcondn(i) variable in waves 1-9 and hcondnew(i) in waves 10 onwards.
-	hcond in wave 1 and new entrants for succeeding waves, hcondn in wave 1-9, hcondnew in wave 10-13

For example, hcond1-19 has 1, 3-13 waves, and it's for new interviewees only. While hcondn1-19 have 2-9 waves asking the existing interviewees about newly devloped conditions, and hcondnew1-19 have wave 9-13 for the same questions. 
hcond21 and hcondnew21 only exist from wave 10-13, while hcondnew22 only exist from 10-13. 

'dcsedfl_dv' is death data, but it's onyl 50% accurate when it comes to health mortality

To check whether the long panel dataset you created has all the correct values, including the special codes like missing values, proxy, refusal, etc., you can use pandas to display the unique values for each variable. This way, you can verify that all expected values are present in the dataset.
**Here's a script that:**
- Loads the long panel dataset.
- Displays the unique values for each variable.
- Checks for the presence of the specified special codes.

In [3]:
import pandas as pd
import os

# Load the long panel data from the Stata file
long_panel_data_path = '/Users/gavinqu/Desktop/School/Dissertation/EssexDissertation/Data/long_panel_ukhls_hcond_data.dta'
long_panel_data = pd.read_stata(long_panel_data_path)

# Define the special codes to check
special_codes = {
    'missing': -9,
    'proxy': -7,
    'refusal': -2,
    'don\'t know': -1,
    'not mentioned': 0,
    'mentioned': 1
}

# Function to check special codes in each variable
def check_special_codes(df, special_codes):
    for column in df.columns:
        if column not in ['pidp', 'wave', 'variable']:
            unique_values = df[column].unique()
            print(f"Unique values in column '{column}': {unique_values}")
            for code_name, code_value in special_codes.items():
                if code_value in unique_values:
                    print(f"  {code_name} ({code_value}) is present in column '{column}'")
                else:
                    print(f"  {code_name} ({code_value}) is NOT present in column '{column}'")
        else:
            unique_values = df[column].unique()
            print(f"Unique values in column '{column}': {unique_values}")

# Check for special codes in the long panel dataset
check_special_codes(long_panel_data, special_codes)

Unique values in column 'pidp': [  68001367   68004087   68006127 ... 1644552890 1644675410 1649095330]
Unique values in column 'wave': ['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm']
Unique values in column 'variable': ['a_hcond11' 'a_hcond13' 'a_hcond5' 'a_hcond7' 'a_hcond1' 'a_hcond15'
 'a_hcond17' 'a_hcond9' 'a_hcond10' 'a_hcond14' 'a_hcond16' 'a_hcond12'
 'a_hcond8' 'a_hcond3' 'a_hcond6' 'a_hcond2' 'a_hcond4' 'b_hcondn9'
 'b_hcondn1' 'b_hcondn6' 'b_hcondn11' 'b_hcondn10' 'b_hcondn8' 'b_hcondn4'
 'b_hcondn7' 'b_hcondn17' 'b_hcondn14' 'b_hcondn2' 'b_hcondn3' 'b_hcondn5'
 'b_hcondn15' 'b_hcondn16' 'b_hcondn13' 'b_hcondn12' 'c_hcondn17'
 'c_hcondn10' 'c_hcondn1' 'c_hcond11' 'c_hcond14' 'c_hcond3' 'c_hcondn13'
 'c_hcondn15' 'c_hcond17' 'c_hcondn4' 'c_hcond1' 'c_hcond7' 'c_hcondn3'
 'c_hcondn11' 'c_hcondn14' 'c_hcondn5' 'c_hcond5' 'c_hcond4' 'c_hcond9'
 'c_hcondn12' 'c_hcondn2' 'c_hcond16' 'c_hcondn16' 'c_hcond6' 'c_hcond13'
 'c_hcond15' 'c_hcond8' 'c_hcond2' 'c_hcondn9' 'c_hcond1

### New approach to include the disdif and hcond along with age and death value in the long panel format: 
1. Load the xhhrel.dta file to get the death information.
2. Merge the death information with the main dataset.
3. Load the disdif variables from each wave and combine them with the existing health condition data.

In [3]:
import pandas as pd
import os

# Base directory containing the data files
base_dir = '/Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls'

# List of base variable names to extract, including 'pidp'
base_hcond_variables = [
    'pidp', 'hcond1', 'hcond2', 'hcond3', 'hcond4', 'hcond5', 'hcond6', 'hcond7',
    'hcond8', 'hcond9', 'hcond10', 'hcond11', 'hcond12', 'hcond13', 'hcond14',
    'hcond15', 'hcond16', 'hcond17', 'hcond21', 'hcond22',
    'hcondn1', 'hcondn2', 'hcondn3', 'hcondn4', 'hcondn5', 'hcondn6', 'hcondn7',
    'hcondn8', 'hcondn9', 'hcondn10', 'hcondn11', 'hcondn12', 'hcondn13', 'hcondn14',
    'hcondn15', 'hcondn16', 'hcondn17', 'hcondnew1', 'hcondnew2',
    'hcondnew3', 'hcondnew4', 'hcondnew5', 'hcondnew6', 'hcondnew7', 'hcondnew8',
    'hcondnew10', 'hcondnew11', 'hcondnew12', 'hcondnew13', 'hcondnew14', 'hcondnew15',
    'hcondnew16'
]

base_disdif_variables = [f'disdif{i}' for i in range(1, 12)]
age_variable = 'age_dv'

# Combine all variable names to extract
base_variables = ['pidp'] + base_hcond_variables[1:] + base_disdif_variables + [age_variable]

# Wave prefixes from 'a' to 'm'
wave_prefixes = [chr(i) for i in range(ord('a'), ord('n'))]

# Function to load and filter wave data
def load_wave_data(wave_prefix, base_dir, base_variables):
    file_path = os.path.join(base_dir, f'{wave_prefix}_indresp.dta')
    if os.path.exists(file_path):
        print(f"Loading data from {file_path}")
        wave_data = pd.read_stata(file_path, convert_categoricals=False)
        
        # Construct the actual variable names for the current wave
        wave_variables = [f'{wave_prefix}_{var}' if var != 'pidp' else var for var in base_variables]
        
        # Find the intersection of desired variables and available columns
        available_columns = set(wave_variables).intersection(wave_data.columns)
        print(f"Available columns in {wave_prefix}: {available_columns}")
        
        # Select only the available columns
        if available_columns:
            selected_data = wave_data[list(available_columns)].copy()
            selected_data['wave'] = wave_prefix
            return selected_data
    return None

# List to store data from each wave
all_waves_data = []

# Loop through wave prefixes
for prefix in wave_prefixes:
    wave_data = load_wave_data(prefix, base_dir, base_variables)
    if wave_data is not None:
        all_waves_data.append(wave_data)

# Combine all waves into a single DataFrame
if all_waves_data:
    combined_data = pd.concat(all_waves_data, ignore_index=True)

    # Load the death information from xhhrel.dta
    death_file_path = os.path.join(base_dir, 'xhhrel.dta')
    death_data = pd.read_stata(death_file_path, convert_categoricals=False)

    # Select the pidp and death column
    death_data = death_data[['pidp', 'dcsedfl_dv']]

    # Merge the death information with the combined data
    combined_data = combined_data.merge(death_data, on='pidp', how='left')

    # Save the combined data with death information to a Stata file
    output_dir = '/Users/gavinqu/Desktop/School/Dissertation/EssexDissertation/Data'
    os.makedirs(output_dir, exist_ok=True)
    combined_data_path = os.path.join(output_dir, 'combined_ukhls_hcond_disdif_death_data.dta')
    combined_data.to_stata(combined_data_path, write_index=False)

    # Reshape the DataFrame into long panel format
    id_vars = ['pidp', 'wave']  # Identifier variables
    value_vars = [col for col in combined_data.columns if col not in id_vars and col not in ['dcsedfl_dv']]  # All other columns

    # Melt the DataFrame
    long_panel_data = pd.melt(combined_data, id_vars=id_vars, value_vars=value_vars,
                              var_name='variable', value_name='value')

    # Save the long panel data to a Stata file
    long_panel_data_path = os.path.join(output_dir, 'long_panel_ukhls_hcond_disdif_death_data.dta')
    long_panel_data.to_stata(long_panel_data_path, write_index=False)

    # Display the first few rows of the long panel data
    print(long_panel_data.head())

    print(f"Combined data saved to {combined_data_path}")
    print(f"Long panel data saved to {long_panel_data_path}")
else:
    print("No data was loaded. Please check the file paths and variable names.")

Loading data from /Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/a_indresp.dta
Available columns in a: {'a_disdif2', 'a_disdif6', 'pidp', 'a_disdif11', 'a_hcond12', 'a_disdif9', 'a_hcond15', 'a_hcond5', 'a_hcond13', 'a_disdif1', 'a_hcond14', 'a_disdif4', 'a_hcond11', 'a_hcond8', 'a_disdif10', 'a_hcond2', 'a_disdif7', 'a_hcond7', 'a_hcond9', 'a_disdif3', 'a_hcond1', 'a_hcond6', 'a_hcond16', 'a_hcond4', 'a_disdif5', 'a_hcond10', 'a_hcond17', 'a_disdif8', 'a_hcond3', 'a_age_dv'}
Loading data from /Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/b_indresp.dta
Available columns in b: {'b_disdif11', 'b_hcondn13', 'pidp', 'b_disdif1', 'b_disdif4', 'b_disdif9', 'b_hcondn12', 'b_hcondn15', 'b_disdif2', 'b_hcondn8', 'b_disdif7', 'b_hcondn16', 'b_hcondn5', 'b_disdif3', 'b_hcondn17', 'b_hcondn11', 'b_hcondn14', 'b_disdif6', 'b_hcondn2', 'b_age_dv', 'b_hcondn4', 'b_hcondn6', 'b_hcondn1', 'b_hcondn3', 'b_disdif8', 'b_hcondn10', 'b_h

/var/folders/n2/8hz3y3r90rj63gkzgrl1hwg40000gn/T/ipykernel_92328/170927490.py:34: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  wave_data = pd.read_stata(file_path, convert_categoricals=False)


Available columns in c: {'c_hcondn5', 'c_hcond10', 'c_hcond17', 'c_disdif6', 'pidp', 'c_hcondn12', 'c_disdif4', 'c_disdif3', 'c_disdif10', 'c_hcondn1', 'c_hcond14', 'c_hcondn11', 'c_hcondn16', 'c_disdif11', 'c_hcond5', 'c_age_dv', 'c_hcond1', 'c_hcond2', 'c_hcond6', 'c_disdif5', 'c_hcondn10', 'c_hcond13', 'c_hcondn13', 'c_disdif7', 'c_disdif8', 'c_disdif2', 'c_hcondn2', 'c_disdif1', 'c_hcondn6', 'c_disdif9', 'c_hcond11', 'c_hcondn14', 'c_hcond15', 'c_hcondn3', 'c_hcond4', 'c_hcond8', 'c_hcond16', 'c_hcond3', 'c_hcond12', 'c_hcondn4', 'c_hcondn8', 'c_hcondn17', 'c_hcond9', 'c_hcondn9', 'c_hcondn15', 'c_hcondn7', 'c_hcond7'}
Loading data from /Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/d_indresp.dta
Available columns in d: {'d_disdif11', 'd_hcond7', 'd_hcond13', 'd_age_dv', 'd_hcond4', 'pidp', 'd_disdif9', 'd_hcond3', 'd_hcond14', 'd_hcond1', 'd_disdif2', 'd_hcondn8', 'd_hcondn1', 'd_hcondn5', 'd_hcondn7', 'd_hcondn14', 'd_disdif3', 'd_hcondn9', 'd_h

/var/folders/n2/8hz3y3r90rj63gkzgrl1hwg40000gn/T/ipykernel_92328/170927490.py:34: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  wave_data = pd.read_stata(file_path, convert_categoricals=False)


Available columns in e: {'e_hcond5', 'e_hcondn11', 'e_hcond13', 'pidp', 'e_hcond4', 'e_hcond12', 'e_disdif11', 'e_hcond7', 'e_disdif9', 'e_hcondn14', 'e_hcondn3', 'e_hcondn6', 'e_disdif3', 'e_age_dv', 'e_disdif7', 'e_hcondn7', 'e_hcond6', 'e_disdif8', 'e_hcondn5', 'e_disdif1', 'e_hcond10', 'e_hcondn15', 'e_hcondn1', 'e_disdif4', 'e_hcond9', 'e_disdif6', 'e_hcondn17', 'e_hcond1', 'e_hcond11', 'e_hcondn16', 'e_hcondn12', 'e_hcond14', 'e_hcondn9', 'e_disdif5', 'e_hcond15', 'e_hcondn8', 'e_hcondn10', 'e_hcond8', 'e_hcond2', 'e_disdif10', 'e_hcond17', 'e_hcond3', 'e_hcond16', 'e_hcondn4', 'e_disdif2', 'e_hcondn13', 'e_hcondn2'}
Loading data from /Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/f_indresp.dta
Available columns in f: {'f_hcondn4', 'f_hcond4', 'pidp', 'f_hcondn16', 'f_disdif5', 'f_hcondn9', 'f_hcondn10', 'f_hcondn11', 'f_hcond7', 'f_disdif6', 'f_hcond16', 'f_hcondn1', 'f_hcondn13', 'f_disdif9', 'f_hcondn8', 'f_hcondn6', 'f_disdif4', 'f_age_dv', 